In [6]:
import numpy
import bempp.api
from bempp.api.operators.boundary import sparse, laplace, modified_helmholtz
from scipy.sparse.linalg import gmres

In [7]:
def rhs_fun(x, n, domain_index,result):
    global phi_static
    result[0] = phi_static[0]
    
def generate_grid(filename):
    
    face = open(filename+'.face','r').read()
    vert = open(filename+'.vert','r').read()

    faces = numpy.vstack(numpy.char.split(face.split('\n')[0:-1]))[:,:3].astype(int) - 1
    verts = numpy.vstack(numpy.char.split(vert.split('\n')[0:-1]))[:,:3].astype(float)

    grid = bempp.api.grid_from_element_data(verts.transpose(), faces.transpose())
    
    N = grid.leaf_view.entity_count(0)
    elements = list(grid.leaf_view.entity_iterator(0))
    area = numpy.zeros(N)

    # remove zero areas
    for i in range(N):
        area[i] = elements[i].geometry.volume

    area_nonzero = numpy.where(area>1e-12)[0]
    
    faces_nonzero = faces[area_nonzero,:]
    
    grid = bempp.api.grid_from_element_data(verts.transpose(), faces_nonzero.transpose())
    
    return grid

def energy_mol(mesh_diel, mesh_stern, phi_static, eps_s):

    grid_diel = generate_grid(mesh_diel)
    grid_stern = generate_grid(mesh_stern)

    space_diel = bempp.api.function_space(grid_diel, "DP", 0)
    space_stern = bempp.api.function_space(grid_stern, "DP", 0)
    
    N_diel = grid_diel.leaf_view.entity_count(0)
    N_stern = grid_stern.leaf_view.entity_count(0)
    
    elements_d = list(grid_diel.leaf_view.entity_iterator(0))
    elements_s = list(grid_stern.leaf_view.entity_iterator(0))

    phis_grid_fun = bempp.api.GridFunction(space_diel, fun=rhs_fun)

    rhs = numpy.concatenate([eps_s*phis_grid_fun.coefficients, 
                      numpy.zeros(N_stern)])

    M11   = laplace.single_layer(space_diel, space_diel, space_diel) 
    M12   = laplace.single_layer(space_stern, space_diel, space_diel) 
    M21   = laplace.single_layer(space_diel, space_stern, space_stern)
    M22   = laplace.single_layer(space_stern, space_stern, space_stern) 

    blocked = bempp.api.BlockedOperator(2, 2)
    blocked[0,0] = M11
    blocked[0,1] = M12
    blocked[1,0] = M21
    blocked[1,1] = M22
    op_discrete = blocked.strong_form()
    
    sigma, info = gmres(op_discrete, rhs, tol=1e-5, maxiter=500, restart = 1000)
    
    if info>0:
        print 'Not converged, %i iterations'%info
    elif info<0:
        print 'Solver diverges'
    
    sigma_d = sigma[:N_diel]
    sigma_s = sigma[N_diel:]

    
    qe = 1.60217662e-19
    Rw = 1.4
    Na = 6.0221409e23
    eps_0 = 8.854187817e-12
    
    #water_charge = 0.83*qe/(numpy.pi*Rw**2) * Na*qe*1e10/(1000*eps_0)
    water_dipole = 6.17e-30/(1e-10)/((2*Rw)**2) * Na*qe*1e10/(1000*eps_0)
    print water_dipole
    print sigma_s[:5]*1.4
    print max(abs(sigma_s*1.4))
    '''
    test_charge = numpy.where(abs(sigma_d)*1.4>water_dipole)[0]
    #print abs(sigma_d)
    if len(test_charge)>0:
        print 'Unphysical charge in dielectric surface'
        print len(test_charge)
    '''    
    
    test_charge = numpy.where(abs(sigma_s)*1.4>water_dipole)[0]
    if len(test_charge)>0:
        print 'Unphysical charge in stern surface'
        print len(test_charge)
    
    area_d = numpy.zeros(N_diel)
    for i in range(N_diel):
        area_d[i] = elements_d[i].geometry.volume
    
    energy = 0.5*numpy.sum(area_d*sigma_d*phi_static)

    return energy, float(len(test_charge))/N_stern

In [8]:
def yoon_lenhoff_matrix(mesh, eps_in, eps_ex, kappa):
    
    grid = generate_grid(mesh)
    
    dirichl_space = bempp.api.function_space(grid, "DP", 0)
    neumann_space = bempp.api.function_space(grid, "DP", 0)

    
    identity = sparse.identity(dirichl_space, dirichl_space, dirichl_space)
    slp_in   = laplace.single_layer(neumann_space, dirichl_space, dirichl_space)
    dlp_in   = laplace.double_layer(dirichl_space, dirichl_space, dirichl_space)
    slp_out  = modified_helmholtz.single_layer(neumann_space, dirichl_space, dirichl_space, kappa)
    dlp_out  = modified_helmholtz.double_layer(dirichl_space, dirichl_space, dirichl_space, kappa)

    # Matrix Assembly
    blocked = bempp.api.BlockedOperator(2, 2)
    blocked[0, 0] = 0.5*identity + dlp_in
    blocked[0, 1] = -slp_in
    blocked[1, 0] = 0.5*identity - dlp_out
    blocked[1, 1] = (eps_in/eps_ex)*slp_out
    A = blocked.strong_form()
    
    A_num = bempp.api.as_matrix(A)

    return A_num 


In [48]:
mesh_stern = 'mobley_test/oct_1_yne/surf_d02_stern'
mesh_diel = 'mobley_test/oct_1_yne/surf_d02'

qe = 1.60217662e-19
Na = 6.0221409e23
eps_0 = 8.854187817e-12
eps_s = 1.
eps_p = 1.
kappa = 0.125

A = yoon_lenhoff_matrix(mesh_diel, eps_p, eps_s, kappa)

w, v = numpy.linalg.eig(A)
v0 = v[:,0]
N = len(v0)/2

phi_static_ref = 40 #kJ/mol/e
conv_factor_2 = phi_static_ref / numpy.average(numpy.real(v0[:N]))
print conv_factor
phi_static = v[0,:N] * conv_factor_2



5.72765746582e-05


In [49]:
conv_factor = (1000*eps_0)/(qe**2*Na*1e10)

e, test_charge = energy_mol(mesh_diel, mesh_stern, phi_static, eps_s)
ekj = e * conv_factor
ekcal = ekj/4.184
print ekcal

857.593881087
[  76.84406861 -8.26051755j -325.12423446+34.94992511j
 -147.41572128+15.84676832j -124.97957327+13.43494659j
 -122.71400455+13.19140444j]
2828.73837966
Unphysical charge in stern surface
5
(-0.0210927817541+0.00934558373222j)


In [56]:
numpy.imag(w)

array([ -1.46233681e+00,   1.46233681e+00,  -9.72634681e-01, ...,
        -1.46921231e-17,   2.62317452e-05,  -2.62317452e-05])

In [58]:
numpy.max(numpy.imag(w))

1.4623368133916332